In [84]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from dotenv import load_dotenv
import os

In [85]:
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

### Extract PDFs

In [86]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)
    document = loader.load()
    
    return document

In [87]:
extracted_data = load_pdf('../data')

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 10 65536 (offset 0)
Ignoring wrong pointing object 21 65536 (offset 0)
Ignoring wrong pointing object 25 65536 (offset 0)
Ignoring wrong pointing object 40 65536 (offset 0)
Ignoring wrong pointing object 57 65536 (offset 0)
Ignoring wrong pointing object 73 65536 (offset 0)
Ignoring wrong pointing object 81 65536 (offset 0)
Ignoring wrong pointing object 92 65536 (offset 0)
Ignoring wrong pointing object 103 65536 (offset 0)
Ignoring wrong pointing object 108 65536 (offset 0)
Ignoring wrong pointing object 113 65536 (offset 0)
Ignoring wrong pointing object 118 65536 (offset 0)
Ignoring wrong pointing object 123 65536 (offset 0)
Ignoring wrong pointing object 128 65536 (offset 0)
Ignoring wrong pointing object 133 65536 (offset 0)
Ignoring wrong pointing object 138 65536 (offset 0)
Ignoring wrong pointing object 143 65536 (offset 0)
Ignoring wrong pointing object 148 65536 (offset 0)
Ignoring wrong pointin

### Creating text chunks

In [88]:
def text_split(extracted_data):
    splited_text = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
    text_chunk = splited_text.split_documents(extracted_data)
    
    return text_chunk

In [89]:
text_chunk = text_split(extracted_data)
len(text_chunk)

5862

### Download HuggingFace Embedding

In [90]:
def download_hf_embeddings():
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedding

In [91]:
embedding = download_hf_embeddings()

In [92]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### Pinecone Database

In [93]:
# Create the Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Access the index
index = pc.Index("qchatbot")


In [94]:
texts = [t.page_content for t in text_chunk]

# Create or connect to the vector store
doc_search = PineconeVectorStore.from_texts(
    texts=texts,
    embedding=embedding,
    index_name="qchatbot",
    namespace="default"  # optional
)

### Similarity search

In [96]:
query = "What is reversible gate?"
docs = doc_search.similarity_search(query=query, k=3)
print(docs)

[Document(id='fc161a24-c7ac-4aae-88d9-f6f7a4f5285f', metadata={}, page_content='A reversible gate, is a logic gate where, given the output(s) of the gate, we can\nalways determine what the input(s) was (were). An example is the NOT gate:'), Document(id='910b6f05-c254-4b3f-85e3-c3a99af1e133', metadata={}, page_content='1.5 Reversible Logic Gates 49\nWe can generalize this technique several ways. First, the gate could be a function\nof any number of variables. For example, say we have a gate with three inputsA, B,\nC, and one output f (A,B,C):\nGate\nA\nB\nC\nf(A, B, C)\nThis must be irreversible because there are fewer outputs than inputs. To make it\nreversible, we add a fourth input D that we XOR with f (A,B,C):\nA A\nB B'), Document(id='a1d6dd9b-7952-4e71-a575-db709b6dd67f', metadata={}, page_content='Another way to contrast reversible and irreversible gates is whether information\nis lost. With a reversible gate, no information is lost since we can always recover\nthe inputs from th

In [97]:
template = """
You are a knowledgeable quantum assistant focused strictly on topics related to **quantum mechanics** and **quantum computing**.

Use the provided context to answer the user's question. If the context does not contain relevant information to answer the question, respond with:
**"I'm not sure about that based on the information I have."**

Strictly avoid making up information or answering unrelated topics.

Context:
{context}

Question:
{question}

Answer:
""".strip()

In [98]:
PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
    )

chain_type_kwargs = {'prompt': PROMPT}

In [101]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type='llama',
                    config={'max_new_tokens':512,
                            'temperature':0.8}
                    )

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-681c8210-665c2d81078ed3d55fe997a8;4abe9f9f-249e-4e7a-9d07-bc3962ecf5df)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Invalid username or password.